# (Currently chess only) Dataframe comparing SAE statistics

In [1]:
# Imports
import sys
sys.path.append("../")

from tqdm import tqdm
import pickle
import torch
import einops
from datasets import load_dataset
from typing import Callable, Optional
import math
import os
import itertools
import json
import gc
from joblib import Parallel, delayed

import pandas as pd

from dataclasses import dataclass
import torch
from nnsight import NNsight
import json
from typing import Any
from datasets import load_dataset
from einops import rearrange
from jaxtyping import Int, Float, jaxtyped
from torch import Tensor
import os
from tqdm import tqdm
from transformers import GPT2LMHeadModel
from transformer_lens import HookedTransformer

from circuits.dictionary_learning import AutoEncoder
from circuits.chess_utils import encode_string
from circuits.dictionary_learning import ActivationBuffer
from circuits.dictionary_learning.dictionary import AutoEncoder, GatedAutoEncoder
from circuits.dictionary_learning.trainers.gated_anneal import GatedAnnealTrainer
from circuits.dictionary_learning.trainers.gdm import GatedSAETrainer
from circuits.dictionary_learning.trainers.p_anneal import PAnnealTrainer
from circuits.dictionary_learning.trainers.standard import StandardTrainer
from circuits.dictionary_learning.evaluation import evaluate
from circuits.nanogpt_to_hf_transformers import NanogptTokenizer, convert_nanogpt_model
from circuits.eval_sae_as_classifier import (
    initialize_results_dict, 
    get_data_batch, 
    apply_indexing_function,
    construct_eval_dataset,
    construct_othello_dataset,
    prep_firing_rate_data,
)
from circuits.utils import (
    get_model, 
    get_submodule,
    get_ae_bundle,
    collect_activations_batch,
    get_nested_folders,
    get_firing_features,
    to_device,
    AutoEncoderBundle,
)
import circuits.chess_utils as chess_utils
import circuits.othello_utils as othello_utils
import circuits.othello_engine_utils as othello_engine_utils

from circuits.dictionary_learning.evaluation import evaluate

from IPython import embed

/home/rangell_umass_edu/miniconda3/envs/sae/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Globals

# Dimension key (from https://medium.com/@NoamShazeer/shape-suffixes-good-coding-style-f836e72e24fd):
# F  = features and minibatch size depending on the context (maybe this is stupid)
# B = batch_size
# L = seq length (context length)
# T = thresholds
# R = rows (or cols)
# C = classes for one hot encoding

home_dir = '/project/pi_mccallum_umass_edu/rangell_umass_edu/'
repo_dir = f'{home_dir}/chess-gpt-circuits'

#DEVICE = 'cuda:0'
torch.set_grad_enabled(False)
batch_size = 32
feature_batch_size = batch_size
n_inputs = 2048 # Length of the eval dataset
GAME = "chess" # "chess" or "othello"

models_path = repo_dir + "/models/"

In [3]:
# Load dataset and init game specific variables

if GAME == "chess":
    othello = False

    autoencoder_group_paths = ["/autoencoders/group-2024-05-11/"]
    custom_functions = [chess_utils.board_to_piece_state] #, chess_utils.board_to_pin_state]
    model_name = "adamkarvonen/8LayerChessGPT2"
    # data = construct_eval_dataset(custom_functions, n_inputs, models_path=models_path, device=DEVICE)
    indexing_functions = [chess_utils.find_dots_indices]

elif GAME == "othello":
    othello = True

    autoencoder_group_paths = ["/autoencoders/othello_layer0/"]
    # autoencoder_group_paths = ["autoencoders/othello_layer0/", "autoencoders/othello_layer5_ef4/"]
    custom_functions = [
            # othello_utils.games_batch_no_last_move_to_state_stack_BLRRC,
            othello_utils.games_batch_to_state_stack_BLRRC,
            othello_utils.games_batch_to_state_stack_mine_yours_BLRRC,
        ]
    model_name = "Baidicoot/Othello-GPT-Transformer-Lens"
    # data = construct_othello_dataset(custom_functions, n_inputs, models_path=models_path, device=DEVICE)
    indexing_functions = [None]  # I'm experimenting with these for Othello
else:
    raise ValueError("Invalid game")

## General dataset statistic

This is only dataset dependent, but not SAE dependent and can be calculated once after loading the dataset

In [4]:
def get_true_board_state_counts(pgn_strings, device):
    # Find the true counts of board states over all movers and games in the dataset
    # This could be calculated within the board_to_piece_state evaluation!
    true_board_states_counts = chess_utils.create_state_stacks(pgn_strings, chess_utils.board_to_piece_state)
    true_board_states_counts = chess_utils.state_stack_to_one_hot(
        chess_utils.config_lookup[chess_utils.board_to_piece_state.__name__], 
        device, 
        true_board_states_counts)
    true_board_states_counts = true_board_states_counts.sum(dim=(0,1))
    true_board_states_counts.shape # [RRC]
    return true_board_states_counts

## SAE specific statistic

In [5]:
# Standard evals
def do_standard_evals(results, ae_bundle, device):
    eval_results = evaluate(
        ae_bundle.ae,
        ae_bundle.buffer,
        max_len=ae_bundle.context_length,
        batch_size=min(512, batch_size), # min(n_eval_samples, activation_buffer_out_batch_size) matters
        io="out",
        device=device,
        #n_batches=1000
    )
    for k, v in eval_results.items():
        results[k] = v
    return results

In [6]:
# Evaluation of custom functions
def eval_custom_fn(
    data,
    results,
    n_act_threshs,
    alive_features_F,
    max_activations_F,
    ae_bundle,
    pgn_strings,
    custom_functions,
    encoded_inputs,
    firing_rate_n_inputs,
    indexing_function,
    device
):
    num_features = len(alive_features_F)
    print(
        f"Out of {ae_bundle.dictionary_size} features, on {firing_rate_n_inputs} activations, {num_features} are alive."
    )

    assert len(pgn_strings) >= n_inputs
    assert n_inputs % batch_size == 0

    n_iters = n_inputs // batch_size
    # We round up to ensure we don't ignore the remainder of features
    num_feature_iters = math.ceil(num_features / feature_batch_size)

    thresholds_T = torch.linspace(0, 1, n_act_threshs).to(device)
    thresholds_TF11 = einops.repeat(thresholds_T, "T -> T F 1 1", F=num_features)
    max_activations_1F11 = einops.repeat(max_activations_F, "F -> 1 F 1 1")
    thresholds_TF11 = thresholds_TF11 * max_activations_1F11

    for i in tqdm(range(n_iters), desc="Aggregating statistics"):
        start = i * batch_size
        end = (i + 1) * batch_size
        pgn_strings_BL = pgn_strings[start:end]
        encoded_inputs_BL = encoded_inputs[start:end]
        encoded_inputs_BL = torch.tensor(encoded_inputs_BL).to(device)

        batch_data = get_data_batch(data, pgn_strings_BL, start, end, custom_functions, device)

        all_activations_FBL, encoded_token_inputs = collect_activations_batch(
            ae_bundle, encoded_inputs_BL, alive_features_F
        )

        if indexing_function is not None:
            all_activations_FBL, batch_data = apply_indexing_function(
                pgn_strings[start:end], all_activations_FBL, batch_data, device, indexing_function
            )
        # For thousands of features, this would be many GB of memory. So, we minibatch.
        for feature in range(num_feature_iters):
            f_start = feature * feature_batch_size
            f_end = min((feature + 1) * feature_batch_size, num_features)
            f_batch_size = f_end - f_start

            activations_FBL = all_activations_FBL[
                f_start:f_end
            ]  
            
            thresholds_TF11_slice = thresholds_TF11[:, f_start:f_end, :, :]
            # NOTE: Now F == feature_batch_size
            # Maybe that's stupid and inconsistent and I should use a new letter for annotations
            # I'll roll with it for now


            ### Aggregate batch statistics
            active_indices_TFBL = activations_FBL > thresholds_TF11_slice
            active_counts_TF = einops.reduce(active_indices_TFBL, "T F B L -> T F", "sum")
            off_counts_TF = einops.reduce(~active_indices_TFBL, "T F B L -> T F", "sum")

            results["on_count"][:, f_start:f_end] += active_counts_TF
            results["off_count"][:, f_start:f_end] += off_counts_TF

            for custom_function in custom_functions:
                on_tracker_TFRRC = results[custom_function.__name__]["on"]
                off_tracker_FTRRC = results[custom_function.__name__]["off"]

                boards_BLRRC = batch_data[custom_function.__name__]
                boards_TFBLRRC = einops.repeat(
                    boards_BLRRC,
                    "B L R1 R2 C -> T F B L R1 R2 C",
                    F=f_batch_size,
                    T=thresholds_TF11_slice.shape[0],
                )

                # TODO The next 2 operations consume almost all of the compute. I don't think it will work,
                # but maybe we can only do 1 of these operations?
                active_boards_sum_TFRRC = einops.reduce(
                    boards_TFBLRRC * active_indices_TFBL[:, :, :, :, None, None, None],
                    "T F B L R1 R2 C -> T F R1 R2 C",
                    "sum",
                )
                off_boards_sum_TFRRC = einops.reduce(
                    boards_TFBLRRC * ~active_indices_TFBL[:, :, :, :, None, None, None],
                    "T F B L R1 R2 C -> T F R1 R2 C",
                    "sum",
                )

                on_tracker_TFRRC[:, f_start:f_end, :, :, :] += active_boards_sum_TFRRC
                off_tracker_FTRRC[:, f_start:f_end, :, :, :] += off_boards_sum_TFRRC

                results[custom_function.__name__]["on"] = on_tracker_TFRRC
                results[custom_function.__name__]["off"] = off_tracker_FTRRC

    return results

In [7]:
# Precision, recall, and F1

def get_classification_metrics(results, true_board_states_counts):
    precision_thresh = 0.9
    recall_thresh = 0.01
    f1_thresh = 0.01
    threshs = [precision_thresh, recall_thresh, f1_thresh]
    eps = 1e-8
    R = 8
    C = 13

    true_pos_TFRRC = results['board_to_piece_state']['on'] 
    pos_all_TF = results['on_count']
    true_all_RRC = true_board_states_counts

    precision = true_pos_TFRRC / (pos_all_TF[:, :, None, None, None] +eps) # Note that a feature which always fires (piece present/absent) will have a precision of 1
    recall = true_pos_TFRRC / (true_all_RRC[None, None, :, :, :] +eps)
    f1 = 2 * (precision * recall) / (precision + recall + eps)
    metrics_TFRRC = [precision, recall, f1]

    # Apply threshold
    counts_TFRRC = [metric > thresh for metric, thresh in zip(metrics_TFRRC, threshs)]

    # Drop empty square state counts
    for i in range(len(counts_TFRRC)):
        counts_TFRRC[i][..., 6] = False
    num_board_states = R * R * (C-1)


    ### Fraction of features with high metric on at least one board state
    # High metric for at least one board state
    counts_any_board_TF = [metric.any(dim=(-1,-2,-3)) for metric in counts_TFRRC]

    # Report fraction of all features for count_as_firing_threshold = 0
    frac_any_board_nonzero_1 = [metric[0].float().mean() for metric in counts_any_board_TF]

    # Report fraction of all features for any threshold (choose threshold per feature that maximizes ratio)
    frac_any_board_best_1 = [metric.any(dim=0).float().mean() for metric in counts_any_board_TF]


    ### Fraction of board states that have at least one feature with high metric
    # Check for each board state whether at least one feature has a high metric (using count_as_firing_threshold = 0)
    counts_any_feature_nonzero_RCC = [metric[0].any(dim=0) for metric in counts_TFRRC]

    # Check for each board state whether at least one feature has a high metric (for any count_as_firing threshold)
    counts_any_feature_best_RCC = [metric.any(dim=(0,1)) for metric in counts_TFRRC]

    # Fraction of individual board states at least one feature has a high metric
    frac_any_feature_nonzero_RCC = [metric.sum() / num_board_states for metric in counts_any_feature_nonzero_RCC]
    frac_any_feature_best_RCC = [metric.sum() / num_board_states for metric in counts_any_feature_best_RCC]

    print(frac_any_board_nonzero_1)
    print(frac_any_board_best_1)
    print(frac_any_feature_nonzero_RCC)
    print(frac_any_feature_best_RCC)

    names = ['precision', 'recall', 'f1']
    for i, (name, t) in enumerate(zip(names, threshs)):
        results[f'frac_any_board_per_feature_act-nonzero_{name}-{t}'] = frac_any_board_nonzero_1[i].item()
        results[f'frac_any_board_per_feature_act-best_{name}-{t}'] = frac_any_board_best_1[i].item()
        results[f'frac_any_feature_per_board_act-nonzero_{name}-{t}'] = frac_any_feature_nonzero_RCC[i].item()
        results[f'frac_any_feature_per_board_act-best_{name}-{t}'] = frac_any_feature_best_RCC[i].item()

    return results

## Loop over SAEs

In [ ]:
# Choose aes and indexing functions

# This could be computed once before the loop if adapting loading pgn_strings
# true_board_state_counts = get_true_board_state_counts(pgn_strings)

sweep_results = {}
sweep_result_keys = ['l0', 'frac_variance_explained', 'cossim', 'l2_ratio', 'frac_any_board_per_feature_act-nonzero_precision-0.9', 'frac_any_board_per_feature_act-best_precision-0.9', 'frac_any_feature_per_board_act-nonzero_precision-0.9', 'frac_any_feature_per_board_act-best_precision-0.9', 'frac_any_board_per_feature_act-nonzero_recall-0.01', 'frac_any_board_per_feature_act-best_recall-0.01', 'frac_any_feature_per_board_act-nonzero_recall-0.01', 'frac_any_feature_per_board_act-best_recall-0.01', 'frac_any_board_per_feature_act-nonzero_f1-0.01', 'frac_any_board_per_feature_act-best_f1-0.01', 'frac_any_feature_per_board_act-nonzero_f1-0.01', 'frac_any_feature_per_board_act-best_f1-0.01']

all_autoencoder_paths = []
for group_path in autoencoder_group_paths:
    all_autoencoder_paths += get_nested_folders(repo_dir + group_path) 

param_combinations = list(itertools.product(all_autoencoder_paths, indexing_functions))

#for ae_dir, idx_fn in param_combinations:
#    print(f'ae_dir: {ae_dir}')
#    print(f'idx_fn: {idx_fn}\n')

# autoencoder_path, indexing_function = param_combinations[1]

#for autoencoder_path, indexing_function in tqdm(param_combinations, desc="Autoencoder loop", total=len(param_combinations)):
def compute_results(device_id, autoencoder_path, indexing_function):
    
    device = f"cuda:{device_id}"
    
    torch.cuda.empty_cache()
    gc.collect()
    
    indexing_function_name = "None"
    if indexing_function is not None:
        indexing_function_name = indexing_function.__name__

    print(f"Autoencoder: {autoencoder_path}")
    print(f"Indexing function: {indexing_function_name}")

    # TODO Function below manipulates the loaded data. If we change that, we can load data once and for all at the top of the file
    data = construct_eval_dataset(custom_functions, n_inputs, models_path=models_path, device=device)
    data, ae_bundle, pgn_strings, encoded_inputs = prep_firing_rate_data(
        autoencoder_path, batch_size, models_path, model_name, data, device, n_inputs, othello
    )

    firing_rate_n_inputs = min(int(n_inputs * 0.5), 1000) * ae_bundle.context_length
    # TODO: Custom thresholds per feature based on max activations
    alive_features_F, max_activations_F = get_firing_features(
        ae_bundle, firing_rate_n_inputs, batch_size, device
    )
    true_board_states_counts = get_true_board_state_counts(pgn_strings, device)
    assert true_board_states_counts is not None

    # initialize result dictionary
    n_act_threshs = 10
    results = initialize_results_dict(custom_functions, n_act_threshs, alive_features_F, device)

    # Standard evaluation metrics
    print('do_standard_evals')
    results = do_standard_evals(results, ae_bundle, device)
    
    # delete the buffer
    del ae_bundle.buffer
    
    # Do custom eval metrics
    print('do custom eval metrics')
    results = eval_custom_fn(
        data,
        results,
        n_act_threshs,
        alive_features_F,
        max_activations_F,
        ae_bundle,
        pgn_strings,
        custom_functions,
        encoded_inputs,
        firing_rate_n_inputs,
        indexing_function,
        device
    )

    torch.cuda.empty_cache()
    gc.collect()
    
    results = get_classification_metrics(results, true_board_states_counts)
    ae_name = "-".join(autoencoder_path.split('/')[-3:-1])
    #sweep_results[ae_name] = {}
    #for sweep_key in sweep_result_keys:
    #    sweep_results[ae_name][sweep_key] = results[sweep_key]
    return ae_name, results

results_list = Parallel(n_jobs=16)(delayed(compute_results)(i%4, autoencoder_path, indexing_function) for i, (autoencoder_path, indexing_function) in enumerate(tqdm(param_combinations, desc="Autoencoder loop", total=len(param_combinations))))
#results_list = [compute_results(i%4, autoencoder_path, indexing_function) for i, (autoencoder_path, indexing_function) in enumerate(tqdm(param_combinations, desc="Autoencoder loop", total=len(param_combinations)))]

#autoencoder_path, indexing_function = param_combinations[1]
#compute_results(0, autoencoder_path, indexing_function)


Autoencoder loop:   9%|▉         | 16/180 [00:00<00:01, 106.55it/s]/home/rangell_umass_edu/miniconda3/envs/sae/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/rangell_umass_edu/miniconda3/envs/sae/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Autoencoder: /project/pi_mccallum_umass_edu/rangell_umass_edu//chess-gpt-circuits/autoencoders/group-2024-05-11/group-2024-05-11_chess-gated/trainer4/
Indexing function: find_dots_indices


Autoencoder: /project/pi_mccallum_umass_edu/rangell_umass_edu//chess-gpt-circuits/autoencoders/group-2024-05-11/group-2024-05-11_chess-gated/trainer0/
Indexing function: find_dots_indices


  warnings.warn(


Autoencoder: /project/pi_mccallum_umass_edu/rangell_umass_edu//chess-gpt-circuits/autoencoders/group-2024-05-11/group-2024-05-11_chess-gated/trainer8/
Indexing function: find_dots_indices


...

Aggregating statistics: 100%|██████████| 64/64 [02:57<00:00,  2.78s/it]] 
  warnings.warn(

[tensor(0.4183, device='cuda:3'), tensor(0.2680, device='cuda:3'), tensor(0.3109, device='cuda:3')]
[tensor(0.5988, device='cuda:3'), tensor(0.2680, device='cuda:3'), tensor(0.3124, device='cuda:3')]
[tensor(0.4466, device='cuda:3'), tensor(0.8177, device='cuda:3'), tensor(0.5938, device='cuda:3')]
[tensor(0.5573, device='cuda:3'), tensor(0.8177, device='cuda:3'), tensor(0.7148, device='cuda:3')]
Autoencoder: /project/pi_mccallum_umass_edu/rangell_umass_edu//chess-gpt-circuits/autoencoders/group-2024-05-11/group-2024-05-11_chess-gated_anneal/trainer25/
Indexing function: find_dots_indices


Aggregating statistics:   0%|          | 0/64 [00:00<?, ?it/s]8.44it/s]]]

do_standard_evals
do custom eval metrics
Out of 4096 features, on 256000 activations, 4088 are alive.


do_standard_evals
do custom eval metrics
Out of 8192 features, on 256000 activations, 7986 are alive.


In [9]:
all_autoencoder_paths

['/project/pi_mccallum_umass_edu/rangell_umass_edu//chess-gpt-circuits/autoencoders/group-2024-05-09/chess-gated_trainer0/',
 '/project/pi_mccallum_umass_edu/rangell_umass_edu//chess-gpt-circuits/autoencoders/group-2024-05-09/chess-gated_trainer1/',
 '/project/pi_mccallum_umass_edu/rangell_umass_edu//chess-gpt-circuits/autoencoders/group-2024-05-09/chess-gated_trainer10/',
 '/project/pi_mccallum_umass_edu/rangell_umass_edu//chess-gpt-circuits/autoencoders/group-2024-05-09/chess-gated_trainer11/',
 '/project/pi_mccallum_umass_edu/rangell_umass_edu//chess-gpt-circuits/autoencoders/group-2024-05-09/chess-gated_trainer12/',
 '/project/pi_mccallum_umass_edu/rangell_umass_edu//chess-gpt-circuits/autoencoders/group-2024-05-09/chess-gated_trainer13/',
 '/project/pi_mccallum_umass_edu/rangell_umass_edu//chess-gpt-circuits/autoencoders/group-2024-05-09/chess-gated_trainer14/',
 '/project/pi_mccallum_umass_edu/rangell_umass_edu//chess-gpt-circuits/autoencoders/group-2024-05-09/chess-gated_trainer

In [9]:
for ae_name, results in results_list:
    sweep_results[ae_name] = {}
    for sweep_key in sweep_result_keys:
        sweep_results[ae_name][sweep_key] = results[sweep_key]

In [10]:
df = pd.DataFrame.from_dict(sweep_results, orient='index').sort_values('l0')
df = df.round(3)
df
#df[['frac_variance_explained', 'l0', 'frac_any_board_per_feature_act-nonzero_precision-0.9', 'frac_any_board_per_feature_act-best_precision-0.9', 'frac_any_feature_per_board_act-nonzero_precision-0.9', 'frac_any_feature_per_board_act-best_precision-0.9', 'frac_any_board_per_feature_act-nonzero_recall-0.01', 'frac_any_board_per_feature_act-best_recall-0.01', 'frac_any_feature_per_board_act-nonzero_recall-0.01', 'frac_any_feature_per_board_act-best_recall-0.01', 'frac_any_board_per_feature_act-nonzero_f1-0.01', 'frac_any_board_per_feature_act-best_f1-0.01', 'frac_any_feature_per_board_act-nonzero_f1-0.01', 'frac_any_feature_per_board_act-best_f1-0.01']]
#df.to_csv("small_sae_sweep.csv")

,l0,frac_variance_explained,cossim,l2_ratio,frac_any_board_per_feature_act-nonzero_precision-0.9,frac_any_board_per_feature_act-best_precision-0.9,frac_any_feature_per_board_act-nonzero_precision-0.9,frac_any_feature_per_board_act-best_precision-0.9,frac_any_board_per_feature_act-nonzero_recall-0.01,frac_any_board_per_feature_act-best_recall-0.01,frac_any_feature_per_board_act-nonzero_recall-0.01,frac_any_feature_per_board_act-best_recall-0.01,frac_any_board_per_feature_act-nonzero_f1-0.01,frac_any_board_per_feature_act-best_f1-0.01,frac_any_feature_per_board_act-nonzero_f1-0.01,frac_any_feature_per_board_act-best_f1-0.01
trainer49,2.906,0.978,0.895,0.886,0.337,0.417,0.316,0.438,0.200,0.200,0.762,0.762,0.233,0.234,0.746,0.772
trainer59,2.969,0.979,0.903,0.894,0.295,0.360,0.322,0.449,0.190,0.190,0.814,0.814,0.221,0.223,0.706,0.730
trainer54,3.031,0.979,0.900,0.883,0.345,0.414,0.341,0.457,0.205,0.205,0.745,0.745,0.242,0.243,0.742,0.771
trainer44,3.031,0.980,0.905,0.889,0.332,0.395,0.332,0.448,0.205,0.205,0.809,0.809,0.238,0.239,0.728,0.746
trainer53,4.375,0.982,0.920,0.910,0.335,0.436,0.376,0.505,0.210,0.210,0.807,0.807,0.249,0.251,0.781,0.799
trainer58,4.438,0.981,0.913,0.908,0.309,0.397,0.324,0.492,0.213,0.213,0.818,0.818,0.248,0.248,0.762,0.783
trainer48,4.469,0.982,0.919,0.912,0.335,0.431,0.345,0.470,0.223,0.223,0.809,0.809,0.256,0.258,0.758,0.792
trainer43,4.656,0.981,0.914,0.911,0.294,0.386,0.335,0.496,0.222,0.222,0.818,0.818,0.246,0.247,0.780,0.801
trainer47,6.844,0.985,0.930,0.928,0.306,0.424,0.329,0.467,0.229,0.229,0.818,0.818,0.260,0.261,0.777,0.797
trainer42,6.875,0.985,0.934,0.919,0.305,0.426,0.332,0.495,0.223,0.223,0.818,0.818,0.253,0.253,0.793,0.805


[tensor(0.3307, device='cuda:0'), tensor(0.3707, device='cuda:0'), tensor(0.4022, device='cuda:0')]
[tensor(0.6056, device='cuda:0'), tensor(0.3707, device='cuda:0'), tensor(0.4029, device='cuda:0')]
[tensor(0.3060, device='cuda:0'), tensor(0.8177, device='cuda:0'), tensor(0.6901, device='cuda:0')]
[tensor(0.4492, device='cuda:0'), tensor(0.8177, device='cuda:0'), tensor(0.8047, device='cuda:0')]
[tensor(0.3799, device='cuda:2'), tensor(0.3160, device='cuda:2'), tensor(0.3588, device='cuda:2')]
[tensor(0.5892, device='cuda:2'), tensor(0.3160, device='cuda:2'), tensor(0.3596, device='cuda:2')]
[tensor(0.2982, device='cuda:2'), tensor(0.8177, device='cuda:2'), tensor(0.7630, device='cuda:2')]
[tensor(0.4427, device='cuda:2'), tensor(0.8177, device='cuda:2'), tensor(0.7956, device='cuda:2')]
[tensor(0.3675, device='cuda:2'), tensor(0.2975, device='cuda:2'), tensor(0.3436, device='cuda:2')]
[tensor(0.5673, device='cuda:2'), tensor(0.2975, device='cuda:2'), tensor(0.3436, device='cuda:2')]
